In [1]:
!pip install pyyaml h5py

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.14.0


In [3]:
(train_images, train_labels), (test_images, test_labels) = \
    tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation="relu", input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(
        optimizer="adam",
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy()]
    )
    return model

model = create_model()

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourge outdated usage, and can be ignored

os.listdir(checkpoint_dir)

Epoch 1/10
29/32 [==========================>...] - ETA: 0s - loss: 1.1662 - sparse_categorical_accuracy: 0.6659
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 3s 22ms/step - loss: 1.1378 - sparse_categorical_accuracy: 0.6720 - val_loss: 0.7317 - val_sparse_categorical_accuracy: 0.7740
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.4385 - sparse_categorical_accuracy: 0.8750
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 14ms/step - loss: 0.4192 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.5332 - val_sparse_categorical_accuracy: 0.8380
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2790 - sparse_categorical_accuracy: 0.9289
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 14ms/step - loss: 0.2867 - sparse_categorical_accuracy: 0.9260 - val_loss: 0.4744 - val_sparse_categorical_accuracy: 0.8420
Epoch 4/10
28/32 [=======

['cp.ckpt.index', 'checkpoint', 'cp.ckpt.data-00000-of-00001']

In [6]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3688 - sparse_categorical_accuracy: 0.0800 - 216ms/epoch - 7ms/step
Untrained model, accuracy:  8.00%


In [7]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4166 - sparse_categorical_accuracy: 0.8650 - 94ms/epoch - 3ms/step
Restored model, accuracy: 86.50%


In [8]:
# Include the epoch in the file name (uses "str.format")
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saved the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)

model = create_model()

# Save the weights using the "checkpoint_path" format
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [11]:
print(os.listdir(checkpoint_dir))

latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

['cp-0015.ckpt.index', 'cp-0030.ckpt.data-00000-of-00001', 'cp-0020.ckpt.data-00000-of-00001', 'cp-0040.ckpt.index', 'cp-0035.ckpt.index', 'cp-0030.ckpt.index', 'cp-0050.ckpt.index', 'cp-0035.ckpt.data-00000-of-00001', 'cp-0000.ckpt.index', 'cp-0005.ckpt.data-00000-of-00001', 'cp-0045.ckpt.data-00000-of-00001', 'cp-0015.ckpt.data-00000-of-00001', 'checkpoint', 'cp-0010.ckpt.index', 'cp-0040.ckpt.data-00000-of-00001', 'cp-0050.ckpt.data-00000-of-00001', 'cp-0025.ckpt.index', 'cp-0020.ckpt.index', 'cp-0025.ckpt.data-00000-of-00001', 'cp-0010.ckpt.data-00000-of-00001', 'cp-0045.ckpt.index', 'cp-0000.ckpt.data-00000-of-00001', 'cp-0005.ckpt.index']


'training_2/cp-0050.ckpt'

In [13]:
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 1s - loss: 0.4803 - sparse_categorical_accuracy: 0.8790 - 587ms/epoch - 18ms/step
Restored model, accuracy: 87.90%


In [14]:
# Save the weights
model.save_weights("./checkpoints/my_checkpoint")

model = create_model()

model.load_weights("./checkpoints/my_checkpoint")

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4803 - sparse_categorical_accuracy: 0.8790 - 198ms/epoch - 6ms/step
Restored model, accuracy: 87.90%


In [15]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as as SavedModel.
!mkdir -p saved_model
model.save("saved_model/my_model")

Epoch 1/5
32/32 [==============================] - 1s 7ms/step - loss: 1.1278 - sparse_categorical_accuracy: 0.6700
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4135 - sparse_categorical_accuracy: 0.8900
Epoch 3/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2755 - sparse_categorical_accuracy: 0.9320
Epoch 4/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2154 - sparse_categorical_accuracy: 0.9440
Epoch 5/5
32/32 [==============================] - 0s 8ms/step - loss: 0.1545 - sparse_categorical_accuracy: 0.9680


In [16]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	fingerprint.pb	keras_metadata.pb  saved_model.pb  variables


In [17]:
new_model = tf.keras.models.load_model("saved_model/my_model")

new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4466 - sparse_categorical_accuracy: 0.8530 - 205ms/epoch - 6ms/step
Restored model, accuracy: 85.30%
32/32 [==============================] - 0s 3ms/step
(1000, 10)


In [19]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The ".h5" extension indicates that the model should be saved to HDF5.
model.save("my_model.h5")

Epoch 1/5
32/32 [==============================] - 1s 8ms/step - loss: 1.1623 - sparse_categorical_accuracy: 0.6650
Epoch 2/5
32/32 [==============================] - 0s 8ms/step - loss: 0.4343 - sparse_categorical_accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 7ms/step - loss: 0.2800 - sparse_categorical_accuracy: 0.9390
Epoch 4/5
32/32 [==============================] - 0s 8ms/step - loss: 0.2094 - sparse_categorical_accuracy: 0.9560
Epoch 5/5
32/32 [==============================] - 0s 7ms/step - loss: 0.1511 - sparse_categorical_accuracy: 0.9670


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model("my_model.h5")

new_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 512)               401920    
                                                                 
 dropout_7 (Dropout)         (None, 512)               0         
                                                                 
 dense_15 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4200 - sparse_categorical_accuracy: 0.8590 - 408ms/epoch - 13ms/step
Restored model, accuracy: 85.90%


In [22]:
# MIT License
#
# Copyright (c) 2017 François Chollet
# Copyright (c) 2023 KURAMOCHI Kenta
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.